<a href="https://colab.research.google.com/github/NanoShark/Navigating-Neurological-Realms/blob/main/CSV_IA_Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def calculate_ia(L, R):
    """Calculate Index of Asymmetry (IA)"""
    return abs((L - R) / max(L, R)) * 100

def analyze_asymmetry(data):
    # Extract relevant columns for left and right comparisons
    left_position_cols = ['LeftPosX', 'LeftPosY', 'LeftPosZ']
    right_position_cols = ['RightPosX', 'RightPosY', 'RightPosZ']
    left_rotation_cols = ['LeftRotX', 'LeftRotY', 'LeftRotZ', 'LeftRotW']
    right_rotation_cols = ['RightRotX', 'RightRotY', 'RightRotZ', 'RightRotW']
    left_velocity_cols = ['LeftVelX', 'LeftVelY', 'LeftVelZ']
    right_velocity_cols = ['RightVelX', 'RightVelY', 'RightVelZ']

    # Initialize lists to store IA values for each type of measurement
    ia_positions = []
    ia_rotations = []
    ia_velocities = []

    # Initialize lists to store velocities for calculating the average velocity
    left_velocities = []
    right_velocities = []

    # Calculate IA for positions
    for left_col, right_col in zip(left_position_cols, right_position_cols):
        for index, row in data.iterrows():
            ia_positions.append(calculate_ia(row[left_col], row[right_col]))

    # Calculate IA for rotations
    for left_col, right_col in zip(left_rotation_cols, right_rotation_cols):
        for index, row in data.iterrows():
            ia_rotations.append(calculate_ia(row[left_col], row[right_col]))

    # Calculate IA for velocities and store velocities for average calculation
    for left_col, right_col in zip(left_velocity_cols, right_velocity_cols):
        for index, row in data.iterrows():
            ia_velocities.append(calculate_ia(row[left_col], row[right_col]))
            left_velocities.append(row[left_col])
            right_velocities.append(row[right_col])

    # Calculate mean IA for positions, rotations, and velocities
    mean_ia_positions = np.mean(ia_positions)
    mean_ia_rotations = np.mean(ia_rotations)
    mean_ia_velocities = np.mean(ia_velocities)

    # Calculate average velocities
    avg_left_velocity = np.mean(left_velocities)
    avg_right_velocity = np.mean(right_velocities)

    # Combine results into a dictionary
    asymmetry_results = {
        'mean_ia_positions': mean_ia_positions,
        'mean_ia_rotations': mean_ia_rotations,
        'mean_ia_velocities': mean_ia_velocities,
        'avg_left_velocity': avg_left_velocity,
        'avg_right_velocity': avg_right_velocity
    }

    return asymmetry_results

def determine_asymmetry_threshold(mean_ia, alpha=1.2):
    """Determine the threshold for asymmetry based on persistence of asymmetry in PD"""
    return alpha * mean_ia

def check_patient_asymmetry(asymmetry_results, thresholds):
    """Check if the patient exceeds the asymmetry threshold"""
    results = {}
    for key in asymmetry_results.keys():
        if key in thresholds:
            results[key] = asymmetry_results[key] > thresholds[key]
    return results

def plot_velocity_distribution(left_velocities, right_velocities):
    plt.figure(figsize=(10, 6))
    plt.hist(left_velocities, bins=30, alpha=0.5, label='Left Controller Velocity')
    plt.hist(right_velocities, bins=30, alpha=0.5, label='Right Controller Velocity')
    plt.title('Velocity Distribution')
    plt.xlabel('Velocity (units)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

def plot_asymmetry_over_time(data, ia_positions, ia_rotations, ia_velocities):
    plt.figure(figsize=(12, 8))
    plt.plot(data.index, ia_positions, label='IA Positions')
    plt.plot(data.index, ia_rotations, label='IA Rotations')
    plt.plot(data.index, ia_velocities, label='IA Velocities')
    plt.title('Asymmetry Index Over Time')
    plt.xlabel('Time')
    plt.ylabel('Index of Asymmetry (%)')
    plt.legend()
    plt.show()

def plot_velocity_comparison(data, left_velocity_cols, right_velocity_cols):
    plt.figure(figsize=(12, 8))
    for left_col, right_col in zip(left_velocity_cols, right_velocity_cols):
        plt.plot(data.index, data[left_col], label=f'{left_col} (Left)')
        plt.plot(data.index, data[right_col], label=f'{right_col} (Right)', linestyle='--')
    plt.title('Velocity Comparison Over Time')
    plt.xlabel('Time')
    plt.ylabel('Velocity (units)')
    plt.legend()
    plt.show()

def main():
    # Load the CSV file
    file_path = '/content/gdrive/MyDrive/Colab Notebooks/SwimmerData_20240828_014743.csv'  # Replace with your actual CSV file path
    data = pd.read_csv(file_path)

    # Analyze asymmetry
    asymmetry_results = analyze_asymmetry(data)

    # Determine thresholds based on healthy individuals' data (example values used here)
    healthy_mean_ia_positions = 5  # Example value, replace with actual data
    healthy_mean_ia_rotations = 4  # Example value, replace with actual data
    healthy_mean_ia_velocities = 3  # Example value, replace with actual data

    thresholds = {
        'mean_ia_positions': determine_asymmetry_threshold(healthy_mean_ia_positions),
        'mean_ia_rotations': determine_asymmetry_threshold(healthy_mean_ia_rotations),
        'mean_ia_velocities': determine_asymmetry_threshold(healthy_mean_ia_velocities)
    }

    # Check patient asymmetry
    patient_asymmetry = check_patient_asymmetry(asymmetry_results, thresholds)

    # Output results
    print("Asymmetry Analysis Results:")
    print(asymmetry_results)
    print("Patient Asymmetry Flags:")
    print(patient_asymmetry)

    # Plotting graphs
    left_velocity_cols = ['LeftVelX', 'LeftVelY', 'LeftVelZ']
    right_velocity_cols = ['RightVelX', 'RightVelY', 'RightVelZ']

    plot_velocity_distribution(asymmetry_results['avg_left_velocity'], asymmetry_results['avg_right_velocity'])
    plot_asymmetry_over_time(data, asymmetry_results['mean_ia_positions'], asymmetry_results['mean_ia_rotations'], asymmetry_results['mean_ia_velocities'])
    plot_velocity_comparison(data, left_velocity_cols, right_velocity_cols)

if __name__ == "__main__":
    main()
